<a href="https://colab.research.google.com/github/alexlapshov89/ALapshov_coding/blob/main/math%26ml_14_part02_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендательные системы

Проект выполнен на основе данных соревнования на [Kaggle.com](https://www.kaggle.com/datasets/gspmoreira/articles-sharing-reading-from-cit-deskdrop). В ходе проекта построена простая рекомендательная система на основе данных о популярности статей.

In [52]:
# библиотеки
import pandas as pd
import numpy as np
import math

In [53]:
# загрузка данных
shared_articles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Skillfactory/MATH&ML-14 Рекомендательные системы. Часть 1/data/shared_articles.csv')
users_interactions = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Skillfactory/MATH&ML-14 Рекомендательные системы. Часть 1/data/users_interactions.csv')
print(f'shared_articles ({shared_articles.shape[0]} lines):\n')
display(shared_articles.head(2))
print(f'users_interactions ({users_interactions.shape[0]} lines):\n')
display(users_interactions.head(3))

shared_articles (3122 lines):



,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en


users_interactions (72312 lines):



,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN


In [54]:
# по условию задачи рассматриваем датасет с метками "CONTENT SHARED"
shared_articles = shared_articles[shared_articles['eventType']=='CONTENT SHARED']
# число записей в рассматриваемом датасете
shared_articles.shape[0]

3047

In [55]:
# cловарь весов для реакций клиентов
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}
# признак "eventType" приводится к числовому типу
users_interactions['eventType'] = users_interactions['eventType'].apply(lambda key: event_type[key])
# контрольный вопрос
round(users_interactions['eventType'].mean(),2)

1.24

In [56]:
# число статей, рассмотренных каждым пользователем
users_articles_count = users_interactions.groupby(by='personId')['contentId'].nunique().rename('articles_count')
# подсчет пользователей, рассматревших более 5 статей
users_articles_count[users_articles_count>=5].shape[0]

1140

In [57]:
# оставляем пользователей, которые взаимодействовали с более чем 5 статьями
users_list = list(set(users_articles_count[users_articles_count>=5].index))
mask = users_interactions['personId'].apply(lambda x: True if x in users_list else False)
users_interactions = users_interactions[mask]
# контрольный ворпос
users_interactions.shape[0]

69868

In [58]:
# функция, логарифмирующая сумму весов действий пользователя
def smooth_user_preference(x):
    return math.log(1+x, 2)

# для каждого пользователя группируем статьи и определяем последнюю временную метку и сумму весов действий
grouped = users_interactions.groupby(by=['personId','contentId']).agg({'timestamp': np.max ,'eventType': np.sum})
grouped['eventType'] = grouped['eventType'].apply(smooth_user_preference)
# контрольный вопрос
round(grouped['timestamp'].mean(),2)


1470605340.04

In [59]:
# временная отсечка
time_lim = 1475519545
# обучающая и тестовая выборки
#train_df = users_interactions[users_interactions['timestamp']<time_lim]
#test_df = users_interactions[users_interactions['timestamp']>=time_lim]
train_df = grouped[grouped['timestamp']<time_lim].reset_index()
test_df = grouped[grouped['timestamp']>=time_lim].reset_index()
# контрольный вопрос
train_df.shape[0]

29325

In [60]:
# таблица в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков
final_df = (
    train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df

,true_train,true_test
personId,,
-9223121837663643404,"[-8949113594875411859, -8377626164558006982, -...","[-7423191370472335463, -6872546942144599345, -..."
-9212075797126931087,"[-1995591062742965408, -969155230116728853, 17...",[]
-9207251133131336884,"[-9216926795620865886, -8742648016180281673, -...",[-4029704725707465084]
-9199575329909162940,"[-5361115220834660562, -5002383425685129595, -...","[-3900870368325485697, 5037403311832115000]"
-9196668942822132778,[-721732705314803549],"[-8813724423497152538, -8535131855706279960, -..."
...,...,...
9165571805999894845,"[-5488842573681626972, -3959242148361340089, -...",[]
9187866633451383747,"[-7660505434580831027, -5571606607344218289, -...","[-9039021554659316935, -4925213312961842798, -..."
9191849144618614467,"[-8742648016180281673, -8208801367848627943, -...","[-5605799891597699962, -4029704725707465084]"


In [61]:
# поиск самых популярных статей в обучающей выборке по сумме баллов действий
popularity_articles = train_df.groupby(by='contentId')['eventType'].sum().sort_values(ascending=False)
popularity_articles.iloc[:5]

contentId
-6783772548752091658    231.177195
-133139342397538859     228.024567
-8208801367848627943    189.937683
 8224860111193157980    186.044680
 7507067965574797372    179.094002
Name: eventType, dtype: float64

In [62]:
# рекомендательная система
def recomendations(true_train_list):
  """"
  Функция исключает из рассмотрения статьи, присутствующие в true_train_list,
  и возвращает топ-10 статей из оставшихся.
  Args:   true_train_list - список статей, с которыми пользователь уже знаком
  Return: список из 10 статей

  """
  contentIds = list(set(popularity_articles.index).difference(set(true_train_list)))
  return list(popularity_articles.loc[contentIds].sort_values(ascending=False).iloc[:10].index)

# вводим данные с рекомендациями в датасет
final_df['recomendations'] = final_df['true_train'].apply(recomendations)

In [63]:
# функция подсчета среднего P@10 для всего датасета
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()

round(precision('recomendations'),3)

0.006